In [1]:
from fastai.text import *
import html

In [2]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag
PATH=Path('data/toxic-comments/')

In [11]:
chunksize=24000
n_labels=7
classes=['toxic','severe_toxic','obscene','threat','insult','identity_hate']

# Data Preparation

In [5]:
df_train = pd.read_csv(PATH/'train.csv')
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
df_test = pd.read_csv(PATH/'test.csv')
len(df_test),len(df_train)

(153164, 159571)

In [8]:
np.random.seed(42)
trn_idx = np.random.permutation(len(df_train))
val_idx = np.random.permutation(len(df_test))

trn_texts = df_train.iloc[trn_idx]['comment_text']
val_texts = df_test.iloc[val_idx]['comment_text']
trn_texts.head()

119105    Geez, are you forgetful!  We've already discus...
131631    Carioca RFA \n\nThanks for your support on my ...
125326    "\n\n Birthday \n\nNo worries, It's what I do ...
111256    Pseudoscience category? \n\nI'm assuming that ...
83590     (and if such phrase exists, it would be provid...
Name: comment_text, dtype: object

In [9]:
trn_texts,val_texts = sklearn.model_selection.train_test_split(
    np.concatenate([trn_texts,val_texts]), test_size=0.1)

In [10]:
len(trn_texts), len(val_texts)

(281461, 31274)

In [31]:
def get_df(texts):
    trn_dict = {'text':texts}
    trn_dict.update({cls: [0]*len(texts) for cls in classes})
    return pd.DataFrame(trn_dict, columns=classes+['text'])

df_trn = get_df(trn_texts)
df_val = get_df(val_texts)

len(df_trn),len(df_val)

(281461, 31274)

In [32]:
df_trn.to_csv(PATH/'train_lm.csv', header=False, index=False)
df_val.to_csv(PATH/'test_lm.csv', header=False, index=False)

In [8]:
def get_texts(df, n_lbls=1):
    labels = df.iloc[:,range(n_lbls)].values.astype(np.int64)
    texts = f'\n{BOS} {FLD} 1 ' + df[n_lbls].astype(str)
    for i in range(n_lbls+1, len(df.columns)): texts += f' {FLD} {i-n_lbls} ' + df[i].astype(str)
    texts = list(texts.apply(fixup).values)

    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok, list(labels)